# Malware Analysis & Triage Kit
This notebook performs the initial stages of immediate malware triage.

## How To
Take your malware specimen and drop it into the `dropbox` directory. The notebook will walk you through the stages of initial analysis.

At the end of this process, you will have a triage report in the `saved-specimens` diretory. This report includes findings from initial triage, including the defanged specimen in a password-proteced Zip file and static analysis artifacts.

# Imports and Setup

In [ ]:
# Imports
from hashlib import *
import sys
import os
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from MalwareSample import *
from pprint import pprint
import os.path
from time import sleep

import subprocess
import json
!pip3 install pandas
from pandas import pandas as pandas
from pandas import DataFrame as df
import numpy
import requests
import pprint
import io
import math
import csv
from datetime import date, timedelta
from functools import reduce
#from pandarallel import pandarallel
#pandarallel.initialize()
import time
import urllib
import bz2
import datetime
import lxml
import re
#from tqdm import tqdm
#import requests_async
#import matplotlib.pyplot as plt
import asyncio
#import itables.interactive
#import qgrid
!pip3 install itables
import itables.options as opt
pandas.set_option('display.max_rows', 100000)
pandas.set_option('display.max_columns', 500)
pandas.get_option('display.max_columns')
pandas.set_option('display.width', 10000)


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pandas.set_option('display.max_colwidth', None)
opt.maxBytes = 90000**6


import os
import requests
from bs4 import BeautifulSoup
import time
import re
import io
from zipfile import ZipFile


## Download Samples from Malware Traffic Analysis

This code is used to download all samples for the given years (in script). The files are downloaded into folders by month.  Then the files are moved in to the dropbox folder and the other directories removed.

In [ ]:


EXTENSION_LIST = [
    'doc',
    'xls',
    'ppt',
    'cab',
    'exe',
    'pe',
    'apk',
    'macos',
    'elf',
    'dylib', # Mach-O/Unibin
    'bundle', # Mach-O/Unibin
    'dmg',
    'jar',
    'pdf',
    'eicar',
    'swf'
]


class MaliciousSamples:

    DATE_RE = re.compile('^\d{4}-\d{2}-\d{2}$')
    MALWARE_TRAFFIC_ANALYSIS_URL = 'https://malware-traffic-analysis.net'

    def malware_traffic_analysis(self, 
                                 years=['2022'], 
                                 extensions=EXTENSION_LIST,
                                 save_path='dropbox/'
        ):
        self.extensions = extensions
        date_links = []
        for year in years:
            year_url = '{}/{}/index.html'.format(self.MALWARE_TRAFFIC_ANALYSIS_URL, year)
            year_response = requests.get(year_url)
            if year_response.status_code == 200:
                year_text = year_response.text
                soup = BeautifulSoup(year_text, 'lxml')
                year_links = soup.find_all('a')
                for year_link in year_links:
                    text = year_link.get_text()
                    href = year_link.get('href')
                    if self.DATE_RE.match(text):
                        link = '{}/{}/{}'.format(self.MALWARE_TRAFFIC_ANALYSIS_URL, year, href)
                        date_links.append(link)
        malware_zip_links = []
        for date_url in date_links:
            time.sleep(1)
            date_folder = '/'.join(date_url.split('/')[0:-1])
            folder_path =date_folder.split(self.MALWARE_TRAFFIC_ANALYSIS_URL)[-1].strip()
            folder_path = save_path + folder_path
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            date_response = requests.get(date_url)
            if date_response.status_code == 200:
                date_text = date_response.text
                soup = BeautifulSoup(date_text, 'lxml')
                links = soup.find_all('a')
                for link in links:
                    text = link.get_text().upper()
                    if 'MALWARE' in text and '.ZIP' in text:
                        href = link.get('href')
                        malware_link = '{}/{}'.format(date_folder, href)
                        malware_request = requests.get(malware_link)
                        if malware_request.status_code == 200:
                            contents = malware_request.content
                            filebytes = io.BytesIO(contents)
                            zf = ZipFile(filebytes)
                            if 'all' in self.extensions:
                                zf.extractall(path=folder_path, pwd=b'infected')
                            else:
                                for name in zf.namelist():
                                    for extension in self.extensions:
                                        if name.endswith(extension):
                                            zf.extract(name, folder_path, pwd=b'infected')
                            malware_zip_links.append(folder_path)
        return malware_zip_links

    def test_files(self):
        '''
            If you have SSL decryption enabled on the firewall, use one of the following URLs:
                PE—https://wildfire.paloaltonetworks.com/publicapi/test/pe
                APK—https://wildfire.paloaltonetworks.com/publicapi/test/apk
                MacOSX—https://wildfire.paloaltonetworks.com/publicapi/test/macos
                ELF—https://wildfire.paloaltonetworks.com/publicapi/test/elf
            If you do not have SSL decryption enabled on the firewall, use one of the following URLs instead:
                PE—http://wildfire.paloaltonetworks.com/publicapi/test/pe
                APK—http://wildfire.paloaltonetworks.com/publicapi/test/apk
                MacOSX—http://wildfire.paloaltonetworks.com/publicapi/test/macos
                ELF—http://wildfire.paloaltonetworks.com/publicapi/test/elf
        '''



print(MaliciousSamples().malware_traffic_analysis())
!cp -a --link ../dropbox/2022/* /dropbbox/.  &&  rm -rf ../dropbox/2022/
!rm -rf dropbox/2022

### Check Dropbox and Saved-Specimens

In [ ]:
MalwareSample.check_dir("dropbox")
MalwareSample.check_dir("saved-specimens")
empty = MalwareSample.is_dir_empty("dropbox")
if empty:
    print(r"  \\--> " + recc + "Put some samples in the dropbox!")

### Enumerate Samples in the Dropbox

In [ ]:
samples=!ls dropbox/*
for s in samples:
    print(info + "Sample: " + s)

In [ ]:
sample_obj = [MalwareSample(s) for s in samples]

### Create a Saved Specimen directory for the specimen(s)

In [ ]:
for obj in sample_obj:
    saved_sample_name = MalwareSample.create_specimen_dirs(obj.sample_name)
    obj.saved_sample_name = saved_sample_name

### Defang Sample

In [ ]:
for obj in sample_obj:
    sample_path = MalwareSample.move_and_defang(obj.sample_name, obj.saved_sample_name)
    obj.sample_path = sample_path

---

## File Hashes

### SHA256 Sum

In [ ]:
for obj in sample_obj:
    hash = MalwareSample.get_sha256sum(obj.sample_path, obj.saved_sample_name)
    obj.sha256sum = hash
    print(info + obj.sample_name + ": " + obj.sha256sum)

---

## String Analysis

### StringSifter
StringSifter is a FLARE developed tool that uses an ML model to rank a binary's strings by relevance to malware analysis.

In [ ]:
length = int(input(recc + "Input your desired minimum string length [default is 4, 6-8 is recommended] > "))

In [ ]:
for obj in sample_obj:
    MalwareSample.pull_strings(length, obj.saved_sample_name, obj.sample_path)

## VT Analysis
Submit samples to Virus Total and generate a malicious confidence level.

In [ ]:
VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

In [ ]:
if VT_API_KEY:
    vt = VirusTotalPublicApi(VT_API_KEY)
else:
    print(info + "No VT API Key. Skipping...")

Note: If there are more than 4 samples in the dropbox, hashes are submitted with a sleep of 16 seconds to remain under the public API rate limit. So hit go, grab a beverage of choice, stretch out and relax. This could be a while depending on how many samples you're submitting.

In [ ]:
if VT_API_KEY:
    for obj in sample_obj:
        print(info + obj.sample_name + ":")
        print(r"  \\--> " + info + "SHA256sum: " + obj.sha256sum)
        res = vt.get_file_report(obj.sha256sum)
        conf = malicious_confidence(res)
        print(r"  \\--> " + info + "Confidence level: " + str(conf))
        crit_level =  determine_criticality(conf)
        obj.criticality = crit_level
        

        if len(sample_obj) >= 5:
            sleep(16)
        
else:
    print(info + "No VT API Key. Skipping...")

## Binary Analysis

### Run PeFrame

Run PEFrame tool and output to json file for json parsing

In [ ]:
for s in sample_obj:
    with open("saved-specimens/" + s.saved_sample_name + "/peframe.txt", "w") as filep:
        print("analyzing " + s.sample_name + " with peframe")
        proc = subprocess.run("peframe -j dropbox/" + s.sample_name, stdout=filep,text=True,shell=True)
print("peframe analysis completed")

## Zip and Password Protect

In [ ]:
for obj in sample_obj:
    zip_file = MalwareSample.zip_and_password_protect(obj.sample_path, obj.saved_sample_name)
    MalwareSample.delete_unzipped_sample(obj.sample_path, zip_file)

---

### Debug Object Vars

In [ ]:
for obj in sample_obj:
    pprint(vars(obj))